# 第5部 状態空間モデルとカルマンフィルタ

## パッケージインポート

In [1]:
import datetime
import numpy as np
import pandas as pd
import pmdarima as pm
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import statsmodels.graphics.api as smg
import statsmodels.tsa.api as tsa
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf

# from pydataset import data
from rdatasets import data

%matplotlib inline
# sns.set(font="DejaVu Sans") #seabornを日本語対応

## 第5部-7章 実装：Pythonによる状態空間モデル

In [17]:
# データの準備
df = data('Nile')
df['time'] = pd.to_datetime(df['time'], format='%Y')
df = df.set_index('time')
df

,value
time,
1871-01-01,1120
1872-01-01,1160
1873-01-01,963
1874-01-01,1210
1875-01-01,1160
...,...
1966-01-01,746
1967-01-01,919
1968-01-01,718


In [18]:
class KfLocalLevel:
    def __init__(self, y, mu_pre, P_pre, sigma_w, sigma_v):
        self.y = y
        self.mu_pre = mu_pre
        self.P_pre = P_pre
        self.sigma_w = sigma_w
        self.sigma_v = sigma_v

    def predict(self):
        self.mu_forecast = self.mu_pre
        self.P_forecast = self.P_pre + self.sigma_w
        self.y_forecast = self.mu_forecast
        self.F = self.P_forecast + self.sigma_v

    def filtering(self):
        self.K = self.P_forecast / (self.P_forecast + self.sigma_v)
        self.y_resid = self.y - self.y_forecast
        self.mu_filter = self.mu_forecast + self.K * self.y_resid
        self.P_filter = (1 - self.K) * self.P_forecast
    
    def get_result(self):
        result = pd.DataFrame(
            {'mu_filter': self.mu_filter, 
            'P_filter': self.P_filter, 
            'y_resid': self.y_resid, 
            'F': self.F, 
            'P': self.P}
        )
        return result


In [19]:
# サンプルサイズ
N = len(df)